In [35]:
%pylab inline
%precision 6

Populating the interactive namespace from numpy and matplotlib


/home/f.arbuzov/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['clf']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


'%.6f'

In [36]:
import pandas as pd
import sklearn as skl
import sklearn

In [37]:
#you may download these functions from https://github.com/Apogentus/common
from common.serialization import pickle_load, pickle_save
from common.classes.Struct import Struct
from common.feature_transformations import get_one_hot_encoding
from common.functions import all_nums

In [38]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn import metrics

In [39]:
pd.options.display.max_colwidth=100
np.set_printoptions(linewidth=140,edgeitems=10)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
rcParams['figure.figsize'] = (8.0, 5.0)

In [40]:
Z_train = pd.read_csv('train.csv', skipinitialspace=True, sep=',', encoding='latin', index_col='PostId')

In [41]:
Z_train

,PostCreationDate,OwnerUserId,OwnerCreationDate,ReputationAtPostCreation,OwnerUndeletedAnswerCountAtPostTime,Title,BodyMarkdown,Tag1,Tag2,Tag3,Tag4,Tag5,PostClosedDate,OpenStatus
PostId,,,,,,,,,,,,,,
0,10/31/2011 12:35:12,993291,10/13/2011 09:56:48,16,0,Compiler error 3921? Following a tutorial and can't work out what is causing this error,I am following a tutorial on Function x <http://www.functionx.com/vcnet/xml/readwrite.htm>. I h...,.net,xml,visual-c++,NaN,NaN,11/19/2011 23:40:06,not a real question
1,05/07/2009 15:19:43,82479,03/25/2009 04:27:50,169,8,Is the CareerCup Book Worth the money?,I just couldnt find a decent review of this book http://www.careercup.com/ \r\nHas anyone in thi...,books,career-development,interview-questions,NaN,NaN,01/31/2012 13:10:04,not constructive
3,12/14/2011 11:16:33,1078847,12/03/2011 07:32:14,1,0,Mailing with Java,I am planning to develop a web application in java using struts/servlets/jsp technology.\r\n\r\n...,java,javascript,servlets,dojo,javamail,12/14/2011 15:25:39,not a real question
5,11/22/2011 04:33:54,532638,12/06/2010 16:03:41,98,0,Ctrl+A doesn't work in Firefox for multi select,If there is a multi select control and you press Ctrl+A in Firefox to select all values then it ...,firefox,multi-select,NaN,NaN,NaN,11/22/2011 22:46:24,off topic
6,10/02/2011 21:01:24,975913,10/02/2011 20:54:48,1,0,How to detect this type of array value use php,"I have a array value bellow\r\n\r\n[ [[6,'SPAIN LA LIGA',0],[[843188,'RCD Espanyol','Real Madrid...",php,json,multidimensional-array,NaN,NaN,10/02/2011 22:13:04,not a real question
7,10/15/2010 14:38:02,477112,10/15/2010 14:33:09,1,0,More about rotateLeft method in java,please tell me examples of rotateLeft method of Integer class in java,java,NaN,NaN,NaN,NaN,10/19/2010 01:22:10,not a real question
9,12/01/2011 14:47:56,955612,09/20/2011 19:21:55,11,1,Create package of latest revisions in Github,I have looked for the 'patch' system but I don't think this is what I want. I've also done a lit...,version-control,github,patch,NaN,NaN,NaN,open
12,09/20/2011 00:00:26,328508,04/28/2010 23:54:42,2875,68,Why did MSFT remove XML digital signature support in MSXML6.0?,MSXML5 is the only version of MSXML that supports XML digital signatures. \r\n\r\nDoes anyone h...,xml,digital-signature,msxml,msxml6,msxml5,07/17/2012 19:41:12,not constructive
14,06/20/2010 20:49:57,183639,10/03/2009 08:19:49,6938,392,What is the best way to stop phishing for online banking?,"[Phishing][1] is a very serious problem that we face. However, banks are the biggest targets. W...",security,phishing,NaN,NaN,NaN,06/27/2010 02:56:09,off topic


In [42]:
Z_test = pd.read_csv('test.csv', skipinitialspace=True, sep=',', encoding='latin', index_col='PostId')

In [43]:
train_inds = Z_train.index.values
test_inds = Z_test.index.values

In [44]:
Z=pd.concat([Z_train,Z_test])

/home/f.arbuzov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


In [45]:
del Z_train, Z_test

In [46]:
len(Z), len(train_inds)+len(test_inds)

(103140, 103140)

In [47]:
len(set(train_inds) & set(test_inds)), len(set(train_inds) | set(test_inds))

(0, 103140)

In [48]:
Z.rename(columns={'OpenStatus':'y'}, inplace=True)

In [49]:
X_train = Z.BodyMarkdown[train_inds].values
Y_train = Z.y[train_inds].values 
Y_train=Y_train.astype(str)

X_test = Z.BodyMarkdown[test_inds].values

In [69]:
len(X_train)

63462

In [61]:
len(X_test)

39678

In [68]:
X_all = np.concatenate([X_train, X_test])
X_all.shape

(103140,)

In [51]:
unique(Y_train, return_counts=1)

(array(['not a real question', 'not constructive', 'off topic', 'open', 'too localized'], dtype='<U19'),
 array([16459, 10421, 10634, 23017,  2931]))

In [70]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X_all)
X_train_tf_idf = vectorizer.transform(X_train)
X_test_tf_idf = vectorizer.transform(X_test)

In [71]:
X_train_tf_idf.shape

(63462, 303839)

In [72]:
X_test_tf_idf.shape

(39678, 303839)

In [73]:
sgdc = SGDClassifier(loss='hinge', alpha=0.0001, max_iter=50)
sgdc.fit(X_train_tf_idf, Y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=50, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [74]:
Y_hat_tf_idf = sgdc.predict(X_test_tf_idf)

In [75]:
Y_hat_tf_idf

array(['open', 'open', 'not a real question', 'open', 'not a real question', 'off topic', 'not a real question', 'not a real question',
       'off topic', 'not constructive', ..., 'open', 'open', 'open', 'open', 'open', 'not constructive', 'open', 'open',
       'not constructive', 'open'], dtype='<U19')

In [76]:
unique(Y_hat_tf_idf, return_counts=1)

(array(['not a real question', 'not constructive', 'off topic', 'open', 'too localized'], dtype='<U19'),
 array([ 8948,  4342,  4296, 22082,    10]))

In [24]:
from sklearn.linear_model import SGDClassifier
clf = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', alpha=0.0001, max_iter=50)),
])

In [25]:
clf.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False))])

In [21]:
Y_hat = clf.predict(X_test)

In [22]:
unique(Y_hat, return_counts=1)

(array(['not a real question', 'not constructive', 'off topic', 'open', 'too localized'], dtype='<U19'),
 array([ 8748,  4387,  4399, 22128,    16]))

# Submission

In [23]:
S = pd.DataFrame(Y_hat, columns=['OpenStatus'], index = Z.loc[test_inds].index)

In [24]:
S.head(10)

,OpenStatus
PostId,
2,open
4,open
8,open
10,open
11,not a real question
13,off topic
16,not a real question
20,not a real question
21,off topic


In [25]:
S.to_csv('baseline_solution.csv')  # accuracy>0.49 on test

Hints for further improvement:
 * tune parameters of baseline model
 * use information about title
 * use information about tags
 * use information about user reputation, duration on StackOverflow
 * generate your own features
 * did moderation policy change in 2012? adapt model scores to anticipate change in policy.